<a href="https://colab.research.google.com/github/chinaskinho/Analise-economica-com-Gemini/blob/main/analise_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wbgapi
!pip install google-generativeai

In [33]:

import pandas as pd
import matplotlib.pyplot as plt
import wbgapi as wb
import seaborn as sns
import os
os.environ['GOOGLE_API_KEY'] = 'key' #adicione aqui sua key da api gemini
import google.generativeai as genai
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

Para escolher o país desejado basta usar o código do país no padrão ISO Alpha-3 (por exemplo, "USA" para Estados Unidos, "BRA" para Brasil).

In [ ]:
pais = '' #escolha do país
# extraindo do World Bank os indicares de PIB, desemprego e inflação
indicators = {
    'NY.GDP.MKTP.KD.ZG': 'PIB',
    'SL.UEM.TOTL.ZS': 'Desemprego',
    'FP.CPI.TOTL.ZG': 'Inflação'
}


country = f'{pais}'
data = wb.data.DataFrame(indicators, country, mrv=10) #mrv (most recent values) serve para controlar o número de anos de dados que você deseja obter.

In [ ]:
#tratando dados
data.rename(index={'FP.CPI.TOTL.ZG': 'Inflação', 'SL.UEM.TOTL.ZS': 'Desemprego', 'NY.GDP.MKTP.KD.ZG': 'PIB'}, inplace=True)
data

In [ ]:
# Separando informações por ano
data_dict = {}
def mostrar_indicadores():
  #data_dict = {}
  for year in data.columns:
    print(f"Ano: {year.replace('YR', '')}")  # Remove 'YR'
    data_dict[year] = {}
    for index in data.index:
        print(f"  {index}: {data.loc[index, year]} %")
        data_dict[year][index] = data.loc[index, year]
    print("\n")
    return data_dict

In [ ]:
# Chamando o Gemini para analise
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content(f"Como um professor de macroeconomia, explique com os dados a seguir a situação atual economica do {pais} e a perspextiva para o futuro. {data_dict}")
# Formatando a saída como Markdown
markdown_output = f"""
# Análise Econômica do {pais}

## Dados Utilizados
Aqui estão os dados econômicos dos últimos 10 anos para o {pais}:

{data.to_markdown()}

## Análise do Gemini
{response.text}
"""


print(markdown_output)